In [3]:
pip install ollama

In [4]:
pip install langchain langchain_community beautifulsoup4 chromadb gradio

Note: you may need to restart the kernel to use updated packages.


Ok, now lets do our RAG

In [10]:
import ollama
import bs4

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [12]:
from langchain.document_loaders import WebBaseLoader

In [13]:
loader = WebBaseLoader("https://tanzil.net/trans/en.shakir")

In [18]:
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
splits = text_splitter.split_documents(data)


In [19]:
print(len(splits))

493


Now we use mistral to create the embeddings and store them in chroma

In [36]:
embeddings = OllamaEmbeddings(model="llama3")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [40]:
retriever = vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model='llama3', messages=[{'role': 'system', 'content': 'You are an assistant who uses the context given to interpret Quranic verses and respond to the question crisply'}, {'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']


In [45]:
# Define the RAG chain
def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = format_docs(retrieved_docs)
    #print(retrieved_docs)
    return ollama_llm(question, formatted_context)

In [46]:
result = rag_chain("What are the attributes of Allah according to the Quran?")
print(result)

According to the Quran, some attributes of Allah (God) include:

1. **Originator of the heavens and the earth** (12:101): This attribute emphasizes Allah's creative power and ability to bring into existence all that exists.
2. **Guardian in this world and the hereafter** (12:102): This attribute highlights Allah's watchful care over every individual, guiding them through both their worldly lives and their eternal lives after death.
3. **Creator of signs in the heavens and the earth** (12:105): This attribute underscores Allah's ability to create various signs or miracles in nature that serve as reminders of His power and wisdom.
4. **Light of the heavens and the earth** (24:35): This attribute emphasizes Allah's illuminating presence, guiding people towards truth, righteousness, and spiritual growth.

These attributes convey Allah's creative, watchful, and guiding powers, highlighting His role as the ultimate Source of guidance, wisdom, and peace in both this life and the next.
